In [1]:
from birdy import WPSClient
import requests
import os
from datetime import datetime
from wps_tools import copy_http_content

# Ensure we are in the working directory with access to the data
if os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../../../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps'
osprey = WPSClient(url)

ImportError: cannot import name 'copy_http_content'

In [ ]:
# NBVAL_IGNORE_OUTPUT
osprey.convert?

In [ ]:
# run convert
convert_output = osprey.convert(
    convert_config = "tests/data/configs/convert_local.cfg"
)

In [ ]:
convert_data = Dataset(make_tmp_copy(convert_output))

# Metadata of the output of RVIC convert module
convert_meatadata = {
    "title": convert_data.title,
    "year": convert_data.history.split()[5],
    "month": convert_data.history.split()[2],
    "day": convert_data.history.split()[3],
    "RvicPourPointsFile": convert_data.RvicPourPointsFile,
    "RvicUHFile": convert_data.RvicUHFile,
    "RvicFdrFile": convert_data.RvicFdrFile,
    "RvicDomainFile": convert_data.RvicDomainFile,
}

In [ ]:
convert_config = read_config("tests/data/configs/convert_local.cfg")

# Expected output metadata determined from the input configuration file
expected_metadata = {
    "title": "RVIC parameter file",
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d"),
    "RvicPourPointsFile": convert_config["UHS_FILES"]["STATION_FILE"].split("/")[-1],
    "RvicUHFile": "unknown",
    "RvicFdrFile": "unknown",
    "RvicDomainFile": convert_config["DOMAIN"]["FILE_NAME"].split("/")[-1],
}

assert convert_meatadata == expected_metadata